In [12]:
import numpy as np
import random as random

In [2]:
def obtainRMatrix(m):
    Rsize = size**2
    rMatrix = inicializeRMatrix(num_col,num_fil)
    res=[]
    for i in range(num_fil):
        for j in range(num_col):
            n0 = m[i][j]
            if 0<i<num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0
                rMatrix[m[i][j]][m[i+1][j]]=0
                
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    
            elif(i==0):
                rMatrix[m[i][j]][m[i+1][j]]=0             
                
                if(j==0):
                    rMatrix[m[i][j]][m[i+1][j+1]]=0             
                    rMatrix[m[i][j]][m[i][j+1]]=0   
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0 
                    
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i+1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                    
            elif i==num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0  
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
    return rMatrix

In [3]:
def addObjetives(R,objetives):
    for objetive in objetives:
        for i in range(num_fil**2):
            R[i][objetive] = 100 if (R[i][objetive]==0) else -1       
        R[objetive][objetive] = 100
    return R          

In [4]:
def addLayerToMatrix(matrix, layer, count):
    for i in range(layer):
        
        matrix[layer][i] = int(count)
        count=count+1
        
    for i in range(layer+1):
        
        matrix[layer-i][layer] = int(count)
        count=count+1
        
    return [matrix, count]

In [5]:
def createMatrix(m,tam):
    layer=0
    for i in range(tam):
        [m,layer]=addLayerToMatrix(m,i,layer)
    return m

In [6]:
def inicializeRMatrix(numRows, numCols):
    return np.array([-1 for _ in range((numRows**2)*(numCols**2))]).reshape(numRows**2, numCols**2)

In [7]:
size = int(input('¿Cuál es el tamaño del problema? '))
num_col = size
num_fil = size
states = (num_col*num_fil)**2

¿Cuál es el tamaño del problema? 3


In [8]:
objetivo = int(input('¿Cuál es el estado objetivo?'))

¿Cuál es el estado objetivo?6


In [9]:
a = np.zeros((num_fil,num_col))

In [10]:
a.astype(int)

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [11]:
m = createMatrix(a,size).astype(int)
m

array([[0, 3, 8],
       [1, 2, 7],
       [4, 5, 6]])

In [12]:
print(m)

[[0 3 8]
 [1 2 7]
 [4 5 6]]


In [13]:
rMatrix=obtainRMatrix(m)

In [14]:
rMatrix

array([[-1,  0,  0,  0, -1, -1, -1, -1, -1],
       [ 0, -1,  0,  0,  0,  0, -1, -1, -1],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1, -1, -1, -1,  0,  0],
       [-1,  0,  0, -1, -1,  0, -1, -1, -1],
       [-1,  0,  0, -1,  0, -1,  0,  0, -1],
       [-1, -1,  0, -1, -1,  0, -1,  0, -1],
       [-1, -1,  0,  0, -1,  0,  0, -1,  0],
       [-1, -1,  0,  0, -1, -1, -1,  0, -1]])

In [15]:
rMatrix=addObjetives(rMatrix,[objetivo])
rMatrix

array([[ -1,   0,   0,   0,  -1,  -1,  -1,  -1,  -1],
       [  0,  -1,   0,   0,   0,   0,  -1,  -1,  -1],
       [  0,   0,  -1,   0,   0,   0, 100,   0,   0],
       [  0,   0,   0,  -1,  -1,  -1,  -1,   0,   0],
       [ -1,   0,   0,  -1,  -1,   0,  -1,  -1,  -1],
       [ -1,   0,   0,  -1,   0,  -1, 100,   0,  -1],
       [ -1,  -1,   0,  -1,  -1,   0, 100,   0,  -1],
       [ -1,  -1,   0,   0,  -1,   0, 100,  -1,   0],
       [ -1,  -1,   0,   0,  -1,  -1,  -1,   0,  -1]])

In [16]:
qMatrix = np.zeros((size**2,size**2)).astype(int)
print(qMatrix)

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


Algoritmo de aprendizaje 
-----------------------------------------------------------

In [17]:
def acciones ( matriz, estado ):
    fila_estado = matriz [ estado , : ]
    acciones = [ ]
    for n in range ( len ( fila_estado ) ):
        if (fila_estado [ n ] != -1):
            acciones.append(n)
    return (acciones)

In [18]:
def maxQValue(Q, R, estado):
    #obtener los estados accesibles desde la matriz R
    rRow = R[estado]
    qRow = Q[estado]
    possibleStatesIndex = np.where(rRow != -1)
    #obtener los valores en la matriz Q
    qLista=qRow[possibleStatesIndex]
    #devolver el máximo
    maxValue=np.amax(qLista)
    return maxValue

In [11]:
def entrenarQ ( Q, R, Gamma, objetivo, num_episodios, alpha, epsilon ):
    #Convertir los valores alpha y epsilon a decimales
    alpha = Decimal( alpha )
    epsilon = Decimal( epsilon )
    while num_episodios > 0:
        estado = random.randint ( 0, num_fil**2 - 1 )
        while ( estado != objetivo ):
            #Selección aleatoria del número que va a ser comparado con epsilon
            numero_aleatorio = random.uniform( 0, 1 ) 
            if numero_aleatorio < epsilon:
                #Número aleatorio < epsilon --> Acción aleatoria
                accion_elegida = random.choice ( acciones ( R, estado ) )
            else:
                #Número aleatorio >= epsilon --> Seleccionamos la acción con el máximo valor en la tabla Q
                qRow = Q [ estado ]
                maxValue = np.max( qRow )
                maxValueIndex = np.where( qRow == maxValue )
                accion_elegida = maxValueIndex[ 0 ] [ 0 ]
            siguiente_estado = accion_elegida
            Q [ estado, accion_elegida ] = R [ estado, accion_elegida ] + Gamma * maxQValue(Q,R,siguiente_estado)
            estado = siguiente_estado
        num_episodios = num_episodios - 1
        epsilon = epsilon * alpha
    return Q

In [5]:
qMatrix = np.zeros((size**2,size**2)).astype(int)
print(qMatrix)

NameError: name 'size' is not defined

In [52]:
alpha = input('Seleccione un parámetro alpha: [0,1]: ')
epsilon = input('Seleccione un parámetro epsilon: [0,1]: ')

Seleccione un parámetro alpha: [0,1]: 4.0
Seleccione un parámetro epsilon: [0,1]: 5.0


In [53]:
entrenarQ(qMatrix, rMatrix, 0.8, objetivo, 10000, alpha, epsilon)

print(qMatrix)

[[  0  64  80  64   0   0   0   0   0]
 [ 64   0  80  64  64  80   0   0   0]
 [ 64  64   0  64  64  80 100  80  64]
 [ 64  64  80   0   0   0   0  80  64]
 [  0  64  80   0   0  80   0   0   0]
 [  0  64  80   0  64   0 100  80   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0  80  64   0  80 100   0  64]
 [  0   0  80  64   0   0   0  80   0]]


In [364]:
qMatrix

array([[  0,  64,  80,  64,   0,   0,   0,   0,   0],
       [ 64,   0,  80,  64,  64,  80,   0,   0,   0],
       [ 64,  64,   0,  64,  64,  80, 100,  80,  64],
       [ 64,  64,  80,   0,   0,   0,   0,  80,  64],
       [  0,  64,  80,   0,   0,  80,   0,   0,   0],
       [  0,  64,  80,   0,  64,   0, 100,  80,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,  80,  64,   0,  80, 100,   0,  64],
       [  0,   0,  80,  64,   0,   0,   0,  80,   0]])

In [365]:
qMatrix=(qMatrix/np.max(qMatrix))*100
print(qMatrix)

[[  0.  64.  80.  64.   0.   0.   0.   0.   0.]
 [ 64.   0.  80.  64.  64.  80.   0.   0.   0.]
 [ 64.  64.   0.  64.  64.  80. 100.  80.  64.]
 [ 64.  64.  80.   0.   0.   0.   0.  80.  64.]
 [  0.  64.  80.   0.   0.  80.   0.   0.   0.]
 [  0.  64.  80.   0.  64.   0. 100.  80.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  80.  64.   0.  80. 100.   0.  64.]
 [  0.   0.  80.  64.   0.   0.   0.  80.   0.]]


In [366]:
rMatrix

array([[ -1,   0,   0,   0,  -1,  -1,  -1,  -1,  -1],
       [  0,  -1,   0,   0,   0,   0,  -1,  -1,  -1],
       [  0,   0,  -1,   0,   0,   0, 100,   0,   0],
       [  0,   0,   0,  -1,  -1,  -1,  -1,   0,   0],
       [ -1,   0,   0,  -1,  -1,   0,  -1,  -1,  -1],
       [ -1,   0,   0,  -1,   0,  -1, 100,   0,  -1],
       [ -1,  -1,   0,  -1,  -1,   0, 100,   0,  -1],
       [ -1,  -1,   0,   0,  -1,   0, 100,  -1,   0],
       [ -1,  -1,   0,   0,  -1,  -1,  -1,   0,  -1]])

Camino eficiente
-----------------------------------------------------------

In [367]:
def findOptimalWay(initialState, Q, objetive):
    #paso1 
    actualState = initialState
    camino = [initialState]
    contador=0
    totalValue=0
    #paso2
    while(actualState!=objetive):
        qRow = Q[actualState]
        maxValue = np.max(qRow)
        maxValueIndex = np.where(qRow == maxValue)
        nextState=maxValueIndex[0][0]
        print("nextState",nextState)
        camino.append(nextState)
        #paso3
        totalValue += maxValue 
        actualState=nextState
        print(actualState)
        contador=contador+1
    return camino, contador, totalValue
    

In [368]:
[camino,contador,totalValue]=findOptimalWay(8, qMatrix, objetivo)

nextState 2
2
nextState 6
6


In [369]:
camino

[8, 2, 6]